# FIFA API: World Cup team stats

#### Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import altair_stiles as altstiles
import numpy as np
import re

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('stiles')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

---

## Get data

#### Read the teams into a dataframe

In [5]:
teams = pd.read_csv("data/processed/world_cup_teams.csv", dtype={"idteam": str})[
    ["idteam", "shortclubname", "abbreviation"]
]

In [6]:
teams.head()

,idteam,shortclubname,abbreviation
0,43922,Argentina,ARG
1,43976,Australia,AUS
2,43935,Belgium,BEL
3,43924,Brazil,BRA
4,43849,Cameroon,CMR


#### Read results from FIFA API for each team

In [7]:
teams_data = []

for team, teamname in zip(teams["idteam"], teams["shortclubname"]):
    team_url = f"https://fdh-api.fifa.com/v1/stats/season/255711/team/{team}.json"
    teams_data.append(
        pd.read_json(team_url)
        .drop([2], axis=1)
        .assign(team_name=teamname, team_id=team)
    )

In [8]:
df = pd.concat(teams_data).reset_index(drop=True)

In [9]:
df = df.rename(columns={0: "stat", 1: "value"})

In [10]:
df["stat"] = (
    df["stat"]
    .astype(str)
    .str.replace(r"(?<=\w)([A-Z])", r" \1", regex=True)
    .str.strip()
    .str.capitalize()
)

In [11]:
df["value"] = df["value"].round(0)

---

In [12]:
df_pivot = df.pivot_table(
    index=["team_name", "team_id"], columns="stat", values="value"
).reset_index()

In [13]:
df_pivot.columns = df_pivot.columns.str.lower().str.replace(" ", "_", regex=False)

In [14]:
df_pivot.sort_values("offsides", ascending=False).head()

stat,team_name,team_id,assists,attempt_at_goal,attempt_at_goal_against,attempt_at_goal_blocked,attempt_at_goal_from_free_kicks,attempt_at_goal_inside_the_penalty_area,attempt_at_goal_inside_the_penalty_area_on_target,attempt_at_goal_off_target,attempt_at_goal_on_target,attempt_at_goal_outside_the_penalty_area,attempt_at_goal_outside_the_penalty_area_on_target,attempted_ball_progressions,attempted_switches_of_play,clean_sheets,completed_ball_progressions,completed_switches_of_play,corners,crosses,crosses_completed,defensive_pressures_applied,direct_defensive_pressures_applied,direct_free_kicks,distance_high_speed_running,distance_high_speed_sprinting,distance_jogging,distance_low_speed_sprinting,distance_walking,distributions_completed_under_pressure,distributions_under_pressure,final_third_entries_reception_central_channel,final_third_entries_reception_inside_left_channel,final_third_entries_reception_inside_right_channel,final_third_entries_reception_left_channel,final_third_entries_reception_right_channel,forced_turnovers,fouls_against,fouls_for,free_kicks,goal_kicks,goalkeeper_defensive_actions_inside_penalty_area,goalkeeper_defensive_actions_outside_penalty_area,goalkeeper_goal_preventions,goals,goals_conceded,goals_conceded_from_attempt_at_goal_against,goals_from_direct_free_kicks,goals_inside_the_penalty_area,goals_outside_the_penalty_area,headed_attempt_at_goal,indirect_free_kicks,linebreaks_attempted,linebreaks_attempted_all_lines,linebreaks_attempted_attacking_and_midfield_line,linebreaks_attempted_attacking_line,linebreaks_attempted_attacking_line_completed,linebreaks_attempted_completed,linebreaks_attempted_defensive_line,linebreaks_attempted_defensive_line_completed,linebreaks_attempted_midfield_and_defensive_line,linebreaks_attempted_midfield_line,linebreaks_attempted_midfield_line_completed,linebreaks_attempted_under_pressure,linebreaks_completed_all_lines,linebreaks_completed_attacking_and_midfield_line,linebreaks_completed_midfield_and_defensive_line,linebreaks_completed_under_pressure,matches_played,offers_to_receive_in_behind,offers_to_receive_in_between,offers_to_receive_in_front,offers_to_receive_inside,offers_to_receive_outside,offers_to_receive_total,offsides,own_goals,passes,passes_completed,penalties,penalties_scored,received_offers_to_receive,receptions_between_midfield_and_defensive_line,receptions_in_behind,receptions_under_direct_pressure,receptions_under_indirect_pressure,receptions_under_no_pressure,receptions_under_pressure,red_cards,substitutions_in,substitutions_out,take_ons_completed,throw_ins,time_played,total_distance,yellow_cards
0,Argentina,43922,8.0,104.0,43.0,8.0,3.0,69.0,36.0,42.0,48.0,35.0,12.0,173.0,44.0,3.0,139.0,38.0,39.0,115.0,30.0,1688.0,332.0,116.0,98287.0,17109.0,331628.0,45687.0,334463.0,1987.0,2647.0,53.0,46.0,42.0,90.0,101.0,494.0,100.0,113.0,123.0,39.0,110.0,1.0,45.0,15.0,8.0,0.0,0.0,14.0,1.0,11.0,7.0,1335.0,64.0,174.0,459.0,340.0,935.0,171.0,96.0,103.0,705.0,499.0,825.0,29.0,174.0,55.0,558.0,7.0,888.0,1710.0,1997.0,2107.0,2488.0,4595.0,23.0,1.0,4375.0,3841.0,5.0,4.0,1827.0,862.0,99.0,137.0,1530.0,2831.0,1667.0,0.0,36.0,36.0,28.0,151.0,742.0,827188.0,16.0
18,Morocco,43872,4.0,61.0,73.0,11.0,4.0,34.0,10.0,28.0,17.0,27.0,7.0,139.0,73.0,4.0,100.0,58.0,12.0,86.0,19.0,2516.0,397.0,84.0,102556.0,17034.0,345468.0,49019.0,298970.0,1084.0,1557.0,24.0,11.0,21.0,62.0,73.0,544.0,97.0,81.0,102.0,71.0,131.0,1.0,77.0,6.0,5.0,0.0,0.0,5.0,1.0,10.0,18.0,997.0,52.0,158.0,433.0,283.0,596.0,104.0,50.0,67.0,460.0,263.0,491.0,14.0,158.0,26.0,290.0,7.0,727.0,1435.0,1358.0,1683.0,1837.0,3520.0,19.0,1.0,2690.0,2219.0,0.0,0.0,1184.0,438.0,60.0,111.0,967.0,1715.0,1078.0,1.0,35.0,35.0,24.0,126.0,696.0,813073.0,9.0
11,France,43946,12.0,101.0,78.0,19.0,2.0,73.0,26.0,42.0,35.0,28.0,9.0,183.0,32.0,1.0,150.0,28.0,38.0,146.0,49.0,1986.0,383.0,84.0,98839.0,17271.0,314479.0,44044.0,328254.0,1398.0,1933.0,29.0,41.0,29.0,113.0,103.0,524.0,69.0,85.0,98.0,65.0,137.0,1.0,80.0,16.0,8.0,0.0,0.0,15.0,1.0,22.0,14.0,1081

In [15]:
df_pivot.columns

Index(['team_name', 'team_id', 'assists', 'attempt_at_goal',
       'attempt_at_goal_against', 'attempt_at_goal_blocked',
       'attempt_at_goal_from_free_kicks',
       'attempt_at_goal_inside_the_penalty_area',
       'attempt_at_goal_inside_the_penalty_area_on_target',
       'attempt_at_goal_off_target', 'attempt_at_goal_on_target',
       'attempt_at_goal_outside_the_penalty_area',
       'attempt_at_goal_outside_the_penalty_area_on_target',
       'attempted_ball_progressions', 'attempted_switches_of_play',
       'clean_sheets', 'completed_ball_progressions',
       'completed_switches_of_play', 'corners', 'crosses', 'crosses_completed',
       'defensive_pressures_applied', 'direct_defensive_pressures_applied',
       'direct_free_kicks', 'distance_high_speed_running',
       'distance_high_speed_sprinting', 'distance_jogging',
       'distance_low_speed_sprinting', 'distance_walking',
       'distributions_completed_under_pressure',
       'distributions_under_pressure',
    

In [16]:
df_slim = df_pivot[
    [
        "team_name",
        "time_played",
        "matches_played",
        "passes",
        "passes_completed",
        "attempt_at_goal",
        "assists",
        "goals",
        "goals_conceded",
        "penalties",
        "red_cards",
        "yellow_cards",
        "offsides",
        "fouls_against",
        "fouls_for",
        "free_kicks",
        "goal_kicks",
        "total_distance",
    ]
].copy()

In [17]:
for c in [
    # "team_name",
    # "time_played",
    # "matches_played",
    "passes",
    "passes_completed",
    "attempt_at_goal",
    "assists",
    "goals",
    "goals_conceded",
    "penalties",
    "red_cards",
    "yellow_cards",
    "offsides",
    "fouls_against",
    "fouls_for",
    "free_kicks",
    "goal_kicks",
    "total_distance",
]:
    df_slim[f"{c}_pg"] = (df_slim[f"{c}"] / df_slim["matches_played"]).round(2)

In [18]:
df_slim[
    [
        "team_name",
        "passes_pg",
        "passes_completed_pg",
        "attempt_at_goal_pg",
        "assists_pg",
        "goals_pg",
        "goals_conceded_pg",
        "penalties_pg",
        "red_cards_pg",
        "yellow_cards_pg",
        "offsides_pg",
        "fouls_against_pg",
        "fouls_for_pg",
        "free_kicks_pg",
        "goal_kicks_pg",
        "total_distance_pg",
    ]
].sort_values("total_distance_pg", ascending=False)

stat,team_name,passes_pg,passes_completed_pg,attempt_at_goal_pg,assists_pg,goals_pg,goals_conceded_pg,penalties_pg,red_cards_pg,yellow_cards_pg,offsides_pg,fouls_against_pg,fouls_for_pg,free_kicks_pg,goal_kicks_pg,total_distance_pg
26,Spain,954.75,881.75,12.75,1.50,2.25,0.75,0.25,0.00,0.50,2.75,10.25,11.25,16.25,6.00,122943.50
7,Croatia,610.29,532.00,11.57,1.14,1.14,1.00,0.00,0.00,1.14,1.57,12.86,12.86,15.14,7.14,122306.86
29,USA,508.25,444.25,11.50,0.75,0.75,1.00,0.00,0.00,1.25,1.25,11.25,9.75,11.50,5.25,122021.50
1,Australia,384.00,301.50,6.25,0.75,1.00,1.50,0.00,0.00,1.75,0.25,13.00,8.50,12.00,9.50,120610.25
15,Japan,398.25,324.50,10.50,1.00,1.25,1.00,0.00,0.00,1.50,2.25,14.50,9.00,11.25,6.25,120164.25
14,IR Iran,327.33,245.00,11.67,0.67,1.33,2.33,0.33,0.00,2.33,2.00,12.33,8.67,10.67,7.67,119635.00
19,Netherlands,560.80,488.40,8.00,1.60,2.00,0.80,0.00,0.20,2.40,1.40,17.40,10.20,13.40,8.00,118453.00
0,Argentina,625.00,548.71,14.86,1.14,2.14,1.14,0.71,0.00,2.29,3.29,14.29,16.14,17.57,5.57,118169.71
12,Germany,633.00,561.67,22.33,1.67,2.00,1.67,0.33,0.00,1.00,4.00,8.67,9.00,13.00,5.00,117135.67
25,Serbia,464.00,388.00,10.33,1.33,1.67,2.67,0.00,0.00,4.00,1.33,14.33,9.00,10.00,7.00,117041.00


---

## Exports

In [19]:
df.to_csv("data/processed/world_cup_team_stats.csv", index=False)
df.query('team_name == "USA"').to_csv(
    "data/processed/world_cup_team_stats_usa.csv", index=False
)

In [20]:
df_pivot.query('team_name == "USA"').to_csv(
    "data/processed/world_cup_team_stats_usa_wide.csv", index=False
)